In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from pathlib import Path
import tensorflow as tf
import json

dataset_root = "D:\David\Škola\RP\dataset"

dataset_filename = "valid_no_dup.json"

with open(Path(dataset_root, dataset_filename)) as json_file:
    raw_json = json.load(json_file)
    
    
i = 1 

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
print("done")

In [2]:
print(_bytes_feature(b'test'))
print(_int64_feature(1))

bytes_list {
  value: "test"
}

int64_list {
  value: 1
}



In [20]:
def process_dataset():
    examples = []
    for outfit in raw_json[:10]:
        set_id = int(outfit["set_id"])
        images = []
        categories = []
        
        for item in outfit["items"]:
            image_path = Path(dataset_root,"images", str(set_id), str(item["index"]) + ".jpg")
            with open(image_path, "rb") as img_file:
                raw_image = img_file.read()
            images.append(raw_image)
            categories.append(item["categoryid"])
            
        outfit_features = {
            "categories": tf.train.FeatureList(feature=[_int64_feature(f) for f in categories]),
            "images": tf.train.FeatureList(feature=[_bytes_feature(f) for f in images])
        }
        
        feature_lists = tf.train.FeatureLists(feature_list=outfit_features)
        
        example = tf.train.SequenceExample(feature_lists=feature_lists)
        examples.append(example.SerializeToString())
        
    return examples
        

examples = process_dataset()
with tf.io.TFRecordWriter("output.tfrecord") as writer:
  for e in examples:
    writer.write(e)

print("The end")

The end


In [21]:
filenames = ["output.tfrecord"]
raw_dataset = tf.data.TFRecordDataset(filenames)

for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)